Project name: 8-Puzzle Solver

In [9]:
import random
import pandas as pd
import heapq

# Goal state
goal_state = ((1, 2, 3), (4, 5, 6), (7, 8, 0))

# Directions for moving the empty space: up, down, left, right
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Manhattan distance heuristic
def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile == 0:
                continue
            goal_i, goal_j = divmod(tile - 1, 3)
            distance += abs(i - goal_i) + abs(j - goal_j)
    return distance

# A* algorithm with pandas DataFrame to track states
def a_star_solver(start_state):
    start_state = tuple(tuple(row) for row in start_state)  # Make the state immutable

    # Priority queue for open list (min-heap)
    open_list = []
    heapq.heappush(open_list, (0 + manhattan_distance(start_state), 0, start_state, []))  # (f, g, state, path)

    # DataFrame to track visited states
    visited = pd.DataFrame(columns=["state", "g", "f", "path"])
    visited = pd.concat([visited, pd.DataFrame([{"state": start_state, "g": 0, "f": manhattan_distance(start_state), "path": []}])], ignore_index=True)

    while open_list:
        f, g, current_state, path = heapq.heappop(open_list)

        # Check if the goal state is reached
        if current_state == goal_state:
            return path

        # Find the position of the empty space (0)
        zero_pos = next((i, j) for i in range(3) for j in range(3) if current_state[i][j] == 0)
        i, j = zero_pos

        # Generate the possible next states
        for di, dj in DIRECTIONS:
            new_i, new_j = i + di, j + dj
            if 0 <= new_i < 3 and 0 <= new_j < 3:
                # Swap the empty space with the tile
                new_state = list(list(row) for row in current_state)
                new_state[i][j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[i][j]
                new_state = tuple(tuple(row) for row in new_state)

                # Check if the new state is already visited
                if not any(new_state == visited_state for visited_state in visited["state"].values):
                    new_g = g + 1
                    new_f = new_g + manhattan_distance(new_state)
                    new_row = pd.DataFrame([{"state": new_state, "g": new_g, "f": new_f, "path": path + [(new_i, new_j)]}])
                    visited = pd.concat([visited, new_row], ignore_index=True)
                    heapq.heappush(open_list, (new_f, new_g, new_state, path + [(new_i, new_j)]))

    return None  # No solution found

# Function to print the puzzle in a user-friendly format
def print_puzzle(state):
    for row in state:
        print(" ".join(str(x) for x in row))
    print()

# Function to allow the user to play the puzzle
def user_play(start_state):
    print("Welcome to the 8-Puzzle game!")

    # Print the initial puzzle state
    print("Initial Puzzle State:")
    print_puzzle(start_state)

    # User input loop to move tiles
    current_state = [list(row) for row in start_state]

    while True:
        print("Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.")
        move = input("Your move: ").strip().lower()

        if move == 'quit':
            print("You chose to quit. The AI will solve the puzzle from the current state.")
            solve_puzzle(current_state)  # Let AI solve from the current state
            break

        if move == 'up':
            di, dj = -1, 0
        elif move == 'down':
            di, dj = 1, 0
        elif move == 'left':
            di, dj = 0, -1
        elif move == 'right':
            di, dj = 0, 1
        else:
            print("Invalid move. Try again.")
            continue

        # Find the position of the empty space (0)
        zero_pos = next((i, j) for i in range(3) for j in range(3) if current_state[i][j] == 0)
        i, j = zero_pos
        new_i, new_j = i + di, j + dj

        # Make sure the move is within bounds
        if 0 <= new_i < 3 and 0 <= new_j < 3:
            # Swap the empty space with the tile
            current_state[i][j], current_state[new_i][new_j] = current_state[new_i][new_j], current_state[i][j]
            print("New Puzzle State:")
            print_puzzle(current_state)
        else:
            print("Invalid move. Try again.")

        # Check if the puzzle is solved
        if tuple(tuple(row) for row in current_state) == goal_state:
            print("Congratulations! You solved the puzzle!")
            break

# Function to automatically solve the puzzle using AI
def solve_puzzle(start_state):
    print("Solving puzzle using AI...")
    path = a_star_solver(start_state)
    if path:
        print("Solution found!")
        for step in path:
            print(f"Move empty space to position {step}")
    else:
        print("No solution found.")

# Function to check if a puzzle state is solvable
def is_solvable(state):
    # Flatten the state and count the number of inversions
    one_d_state = [tile for row in state for tile in row]
    inversions = 0
    for i in range(len(one_d_state)):
        for j in range(i + 1, len(one_d_state)):
            if one_d_state[i] != 0 and one_d_state[j] != 0 and one_d_state[i] > one_d_state[j]:
                inversions += 1
    return inversions % 2 == 0

# Function to generate a random solvable state
def generate_random_state():
    while True:
        state = list(range(9))  # Create a list with values from 0 to 8
        random.shuffle(state)  # Shuffle the list
        state = [tuple(state[i:i + 3]) for i in range(0, 9, 3)]  # Convert to 3x3 tuple state
        if is_solvable(state):
            return state

# Main function to run the game
def main():
    choice = input("Do you want to play the puzzle (P) or solve it using AI (S)? ").strip().lower()

    # Generate a random solvable puzzle state
    start_state = generate_random_state()

    print(f"Initial Puzzle State:\n{start_state}")

    if choice == 'p':
        user_play(start_state)
    elif choice == 's':
        solve_puzzle(start_state)
    else:
        print("Invalid choice. Please choose 'P' to play or 'S' to solve.")

if __name__ == "__main__":
    main()


Do you want to play the puzzle (P) or solve it using AI (S)? p
Initial Puzzle State:
[(1, 5, 2), (8, 4, 0), (6, 7, 3)]
Welcome to the 8-Puzzle game!
Initial Puzzle State:
1 5 2
8 4 0
6 7 3

Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.
Your move: up
New Puzzle State:
1 5 0
8 4 2
6 7 3

Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.
Your move: down
New Puzzle State:
1 5 2
8 4 0
6 7 3

Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.
Your move: left
New Puzzle State:
1 5 2
8 0 4
6 7 3

Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.
Your move: up 
New Puzzle State:
1 0 2
8 5 4
6 7 3

Enter a move: 'up', 'down', 'left', 'right' to move the empty space or 'quit' to stop and solve with AI.
Your move: right
New Puzzle State:
1 2 0
8 5 4
6 7 3

Enter a move: 'up', 'd